In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [2]:
train_folder_yes = 'brain/yes/'
train_folder_no = 'brain/no/'

In [4]:
img_size = 128 
conversion_factor = 0.264 

In [5]:
def process_image(image_path, is_tumor):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image, (img_size, img_size))
    image_array = img_to_array(image_resized) / 255.0  # Normalize to [0, 1]

    area_pixels = 0
    area_mm2 = 0

    if is_tumor:
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Apply thresholding
        _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            # Assume the largest contour is the tumor
            contour = max(contours, key=cv2.contourArea)

            # Calculate area in pixels
            area_pixels = cv2.contourArea(contour)

            # Convert pixel area to mm²
            area_mm2 = area_pixels * (conversion_factor ** 2)

    return image_array, area_pixels, area_mm2


In [6]:
images = []
labels = []
tumor_areas_pixels = []
tumor_areas_mm2 = []

# Process "yes" folder (tumor present)
for filename in os.listdir(train_folder_yes):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(train_folder_yes, filename)
        image_array, area_pixels, area_mm2 = process_image(image_path, is_tumor=True)
        images.append(image_array)
        labels.append('yes')
        tumor_areas_pixels.append(area_pixels)
        tumor_areas_mm2.append(area_mm2)

# Process "no" folder (tumor absent)
for filename in os.listdir(train_folder_no):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(train_folder_no, filename)
        image_array, _, _ = process_image(image_path, is_tumor=False)
        images.append(image_array)
        labels.append('no')
        tumor_areas_pixels.append(0)  # No tumor area
        tumor_areas_mm2.append(0)


In [7]:
X = np.array(images)

In [8]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = to_categorical(y)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # 2 classes: 'yes' and 'no'
])


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
7/7 [==============================] - 7s 729ms/step - loss: 0.7380 - accuracy: 0.6535 - val_loss: 0.6258 - val_accuracy: 0.6667
Epoch 2/10
7/7 [==============================] - 5s 653ms/step - loss: 0.4961 - accuracy: 0.7772 - val_loss: 0.5585 - val_accuracy: 0.7059
Epoch 3/10
7/7 [==============================] - 5s 644ms/step - loss: 0.5229 - accuracy: 0.7970 - val_loss: 0.5485 - val_accuracy: 0.7255
Epoch 4/10
7/7 [==============================] - 5s 655ms/step - loss: 0.4393 - accuracy: 0.8317 - val_loss: 0.5408 - val_accuracy: 0.7451
Epoch 5/10
7/7 [==============================] - 5s 649ms/step - loss: 0.4263 - accuracy: 0.7970 - val_loss: 0.4718 - val_accuracy: 0.7843
Epoch 6/10
7/7 [==============================] - 5s 646ms/step - loss: 0.3448 - accuracy: 0.8614 - val_loss: 0.4957 - val_accuracy: 0.7843
Epoch 7/10
7/7 [==============================] - 5s 650ms/step - loss: 0.2598 - accuracy: 0.9158 - val_loss: 0.4475 - val_accuracy: 0.8235
Epoch 8/10
7/7 [====

In [12]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {accuracy:.2f}")

2/2 [==============================] - 0s 111ms/step - loss: 0.3675 - accuracy: 0.8039
Validation accuracy: 0.80


In [13]:
model.save('brain_model.h5')


In [14]:
def predict_and_measure(image_path):
    image_array, area_pixels, area_mm2 = process_image(image_path, is_tumor=True)
    image_array = np.expand_dims(image_array, axis=0)
    prediction = model.predict(image_array)
    class_idx = np.argmax(prediction)
    predicted_class = label_encoder.inverse_transform([class_idx])[0]

    if predicted_class == 'yes':
        print(f"Tumor detected. Area: {area_mm2:.2f} mm²")
    else:
        print("No tumor detected.")

In [17]:
sample_image = 'brain/yes/Y_1.jpg'
predict_and_measure(sample_image)

Tumor detected. Area: 2707.20 mm²
